In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CS5246/AITA')

Mounted at /content/drive


In [2]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.4 MB/s eta 0:00:00


In [3]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import networkx as nx

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import re
import string
import contractions

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
train = pd.read_csv('train.csv',)
test = pd.read_csv('test.csv')
df = pd.concat([train, test])
df

,flag,content
0,0,My bf (21) and me (22) have been together for ...
1,0,Yesterday afternoon my(19m) aunt's boyfriend b...
2,0,So I've been dating this girl since early dece...
3,0,I like to send my roommate pictures of cute th...
4,0,"Good Afternoon, I'm a 29yo US Venezuelan immig..."
...,...,...
5995,1,AITA for wanting to put our dog down? | My wif...
5996,1,AITA for not going to my sister's funeral beca...
5997,1,AITA? my GF got a leadership position in a col...
5998,1,AITA for hiring somone to do my share of the c...


In [ ]:
train.shape

In [ ]:
test.shape

In [8]:
def read_article(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)
    return len(sentences)

porter_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def tokenize_and_stem(text):
    expanded_text = contractions.fix(text)
    clean_text = re.sub(r'http[s]?://\S+', '', expanded_text)
    tokens = word_tokenize(clean_text)
    lemmas = [wordnet_lemmatizer.lemmatize(token) for token in tokens]
    return lemmas

df['sent_length'] = df['content'].apply(read_article)
df['token_length'] = df['content'].apply(lambda x: len(tokenize_and_stem(x)))
df.describe()

,flag,sent_length,token_length
count,16872.000000,16872.000000,16872.000000
mean,0.399360,21.756520,466.342817
std,0.489781,11.416171,217.605811
min,0.000000,1.000000,9.000000
25%,0.000000,14.000000,309.000000
50%,0.000000,20.000000,438.000000
75%,1.000000,27.000000,603.000000
max,1.000000,194.000000,2714.000000


In [12]:
np.percentile(df['sent_length'], 1)

5.0

In [18]:
def make_sentences(text):
    expanded_text = contractions.fix(text)
    clean_text = re.sub(r'http[s]?://\S+', '', expanded_text)
    sentences = sent_tokenize(clean_text)
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    # Tokenize and remove stopwords
    if stopwords is None:
        stopwords = []
    punctuations = set(string.punctuation)
    punctuations.union(set(['\'', '\`', '\“']))
    sent1 = [word.lower() for word in word_tokenize(sent1) if word.lower() not in stopwords and word not in punctuations]
    sent2 = [word.lower() for word in word_tokenize(sent2) if word.lower() not in stopwords and word not in punctuations]

    # Create a set of all unique words in both sentences
    all_words = list(set(sent1 + sent2))

    # Create vectors to represent each sentence
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Count word occurrences
    for word in sent1:
        vector1[all_words.index(word)] += 1

    for word in sent2:
        vector2[all_words.index(word)] += 1

    # Calculate cosine similarity
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stopwords):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    # Fill the similarity matrix with cosine similarity scores
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)

    return similarity_matrix

def generate_summary(text, top_n=5):
    # Load English stopwords
    stop_words = stopwords.words('english')

    # Tokenize the text into sentences
    sentences = make_sentences(text)
    if len(sentences) > top_n:
        # Generate the sentence similarity matrix
        sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)
        sentence_similarity_matrix = np.nan_to_num(sentence_similarity_matrix, nan=0)

        # Convert the similarity matrix into a graph
        sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)

        try:
            scores = nx.pagerank(sentence_similarity_graph, alpha=0.85, max_iter=100, tol=1e-06)
        except nx.PowerIterationFailedConvergence as e:
            try:
                pr = nx.pagerank(sentence_similarity_graph, alpha=0.85, max_iter=500, tol=1e-04)
            except nx.PowerIterationFailedConvergence as e:
                return ['', '', '', '', '']

        # Sort the sentences by their scores
        ranked_sentences = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)

        # Extract the top sentences as the summary
        summary = [sentence for score, sentence in ranked_sentences[:top_n]]
        return summary
    else:
      while len(sentences) < top_n:
        sentences.append('')
      return sentences

In [22]:
from tqdm import tqdm

#generate top 5 sentences for each reddit post
def split_to_sent(text):
    sent1 = []
    sent2 = []
    sent3 = []
    sent4 = []
    sent5 = []
    pagerank = []

    total_contents = len(text['content'])
    for content in tqdm(text['content'], total=total_contents):
      array_summary = generate_summary(content)
      sent1.append(array_summary[0])
      sent2.append(array_summary[1])
      sent3.append(array_summary[2])
      sent4.append(array_summary[3])
      sent5.append(array_summary[4])
      pagerank.append(' '.join(array_summary))

    text['pagerank'] = pagerank
    text['sent1'] = sent1
    text['sent2'] = sent2
    text['sent3'] = sent3
    text['sent4'] = sent4
    text['sent5'] = sent5

    return text

In [31]:
train = split_to_sent(train)
train.head()

,flag,content,sent1,sent2,sent3,sent4,sent5,pagerank
0,0,My bf (21) and me (22) have been together for ...,Even though personally i do not believe that i...,"Its all started in november, my bf and John li...","For the past months, Emilie has been really me...","While we were playing a game, my bf eliminated...","Now, John told my bf that she was stress and s...",Even though personally i do not believe that i...
1,0,Yesterday afternoon my(19m) aunt's boyfriend b...,He had been ignoring her calls and voicemails ...,I told her that her best bet is to apologize s...,She came over afterwards and told me about this.,The guy bought his cat several catnip toys and...,My dad told me that I should not say something...,He had been ignoring her calls and voicemails ...
2,0,So I've been dating this girl since early dece...,So I have been dating this girl since early de...,On Thursday I got fed up and changed my loving...,On Tuesday I told her about some new things I ...,On Wednesday there were no questions about me.,"I feel sad and mad, I was starting to fell in ...",So I have been dating this girl since early de...
3,0,I like to send my roommate pictures of cute th...,AITA for buying a jewelry box that my roommate...,"My roommate also really loves jewelry, has a j...",They responded saying that they really did wan...,I feel that because they had all day and it to...,I thought the box was really pretty so I arran...,AITA for buying a jewelry box that my roommate...
4,0,"Good Afternoon, I'm a 29yo US Venezuelan immig...","Her mom told her grandma that there was a ""Mex...","I just said ""I am OP, I am here to pick Lucy up"".","Whom I assume was her mom opened it and said ""...","She said ""Ok"" and closed the door, I stood at ...",I could her Lucy's voice but could not really ...,"Her mom told her grandma that there was a ""Mex..."


In [32]:
test = split_to_sent(test)
test.head()

  0%|          | 4/6000 [00:02<1:06:50,  1.50it/s]/usr/local/lib/python3.10/dist-packages/nltk/cluster/util.py:130: RuntimeWarning: invalid value encountered in divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))
  2%|▏         | 111/6000 [00:46<23:52,  4.11it/s]/usr/local/lib/python3.10/dist-packages/nltk/cluster/util.py:130: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - (numpy.dot(u, v) / (sqrt(numpy.dot(u, u)) * sqrt(numpy.dot(v, v))))
100%|██████████| 6000/6000 [32:48<00:00,  3.05it/s]


,flag,content,pagerank,sent1,sent2,sent3,sent4,sent5
0,0,My brother in-law (Sammy) lost his home shortl...,I told him this is between me and my wife but ...,I told him this is between me and my wife but ...,it is normal that girls of the same age borrow...,Zoey complained a lot and I have already asked...,I told my wife and she said she would ask Sloa...,Do not know much about makeup but that is what...
1,0,I'll try to keep this short. I had a [1967 Imp...,I had spent over 11k on that car including new...,I had spent over 11k on that car including new...,i am just so glad they found my car.,A couple months ago I bought my first house th...,I got a call from her mom about 20 minutes ago...,Once again thank you all\n\nUPDATE : went to t...
2,0,I am a high schooler with a weekend job at a c...,"So James came in and was like ""sir is there a ...","So James came in and was like ""sir is there a ...","I did not mean it"" and James was like ""No, we ...","The angry guy started to backtrack, like ""It i...","I started acting real sad, like ""no please do ...","So he was just like ""OP, this is unacceptable,..."
3,0,I am a nurse practitioner and I am the primary...,And my dad says how proud he is of his three o...,And my dad says how proud he is of his three o...,At dinner my mom starts going on about how she...,He was born almost four months premature.,We are getting married in June.,It was almost three months premature and weigh...
4,0,I have a sister that’s 6 years older than me. ...,I told him I would take down the video only wh...,I told him I would take down the video only wh...,My dad told me to take down the video.,From the beginning I told my parents how I am ...,He took my phone then went to my fiancé and as...,My reply was exactly how do you plan to “make ...


In [35]:
train.to_csv('train_with_pagerank.csv', index=False)
test.to_csv('test_with_pagerank.csv', index=False)